In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import cmath

In [138]:
def get_zero_sequence_impedance(sequence_impedance_matrix):
    try:
        return sequence_impedance_matrix[0,0]
    except:
        raise ValueError('sequence_impedance_matrix is not valid.')
        
def get_positive_sequence_impedance(sequence_impedance_matrix):
    try:
        return sequence_impedance_matrix[1,1]
    except:
        raise ValueError('sequence_impedance_matrix is not valid.')
        
def get_negative_sequence_impedance(sequence_impedance_matrix):
    try:
        return sequence_impedance_matrix[2,2]
    except:
        raise ValueError('sequence_impedance_matrix is not valid.')
        
def get_sequence_impedance_matrix(phase_impedance_matrix):
    a=cmath.exp(complex(0,2./3*cmath.pi))
    A=np.array([[complex(1.0,0),complex(1.0,0),complex(1.0,0)],
                [complex(1.0,0),a**2,a],
                [complex(1.0,0),a,a**2]])
    A_inv=1./3.0*np.array([[complex(1.0,0),complex(1.0,0),complex(1.0,0)],
                           [complex(1.0,0),a,a**2],
                           [complex(1.0,0),a**2,a]])
    return np.dot(A_inv,np.dot(phase_impedance_matrix,A))

def kron_reduction(primitive_impedance_matrix):
    zij=primitive_impedance_matrix[:3,:3]
    zin=primitive_impedance_matrix[:3,-1][:,np.newaxis]
    znn=primitive_impedance_matrix[3,3]
    znj=primitive_impedance_matrix[-1,:3]
    return zij-np.dot(zin,np.dot(1.0/znn,znj)[np.newaxis])

def carson_equation_self(ri,GMRi):
    return complex(ri+.0953,.12134*(np.log(1.0/GMRi)+7.93402))

def carson_equation(Dij):
    return complex(.09530, .12134*(np.log(1.0/Dij)+7.93402))

def get_primitive_impedance_matrix(dist_matrix, GMR_list, r_list):
    primitive_impedance_matrix=[]
    n_rows,n_cols=dist_matrix.shape
    for i in range(n_rows):
        primitive_impedance_matrix.append([])
        for j in range(n_cols):
            if i==j:
                primitive_impedance_matrix[-1].append(carson_equation_self(r_list[i],GMR_list[i]))
            else:
                primitive_impedance_matrix[-1].append(carson_equation(dist_matrix[i,j]))
    return np.array(primitive_impedance_matrix)

def get_sequence_impedances(dist_matrix, GMR_list, r_list):
    prim=get_primitive_impedance_matrix(dist_matrix, GMR_list, r_list)
    phase=kron_reduction(prim)
    print phase
    seq=get_sequence_impedance_matrix(phase)
    return get_zero_sequence_impedance(seq), get_positive_sequence_impedance(seq), get_negative_sequence_impedance(seq)

In [139]:
dist_matrix=np.array([[0,2.5,7.0,5.6569],
                      [2.5,0,4.5,4.272],
                      [7.0,4.5,0,5.0],
                      [5.6569,4.272,5.0,0]])
GMR_list=[0.0244,0.0244,0.0244,0.00814]
r_list=[0.306,0.306,0.306,0.592]

In [140]:
get_sequence_impedances(dist_matrix, GMR_list, r_list)

[[ 0.45755083+1.07803538j  0.15594993+0.50167375j  0.15348470+0.38493418j]
 [ 0.15594993+0.50167375j  0.46662762+1.04816333j  0.15800626+0.423648j  ]
 [ 0.15348470+0.38493418j  0.15800626+0.423648j    0.46147239+1.06505793j]]


((0.77351087368443594+1.9372561657931306j),
 (0.30606998553726722+0.6270002376486794j),
 (0.30606998553726722+0.6270002376486794j))

In [126]:
import opendssdirect as dss

In [127]:
dss.dss_lib.DSSPut_Command('new circuit.IEEE13Nodeckt basekv=115 pu=1.0001 phases=3 bus1=SourceBus Angle=30 MVAsc3=20000 MVASC1=21000 ')

''

In [128]:
dss.dss_lib.DSSPut_Command('New Line.650632 Phases=3 Bus1=RG60.1.2.3 Bus2=632.1.2.3 R1=0.30606998553726722 R0=0.77351087368443594 X0=1.9372561657931306 X1=0.6270002376486794 Length=2000 units=ft')

''

In [129]:
dss.Lines.First()

1

In [137]:
dss.utils.lines_to_dataframe()['RMatrix'].values

array([ list([0.09813333333333332, 0.04013333333333333, 0.04013333333333333, 0.04013333333333333, 0.09813333333333332, 0.04013333333333333, 0.04013333333333333, 0.04013333333333333, 0.09813333333333332])], dtype=object)

In [142]:
dss.utils.lines_to_dataframe()['XMatrix'].values

array([ list([0.2153, 0.0947, 0.0947, 0.0947, 0.2153, 0.0947, 0.0947, 0.0947, 0.2153])], dtype=object)

In [146]:
1.0/3.0*(2*0.30606998553726722+0.77351087368443594)

0.4618836149196568

In [147]:
1.0/3.0*(0.77351087368443594-0.30606998553726722)

0.15581362938238957

In [148]:
1.0/3.0*(1.9372561657931306-0.6270002376486794)

0.4367519760481504

In [150]:
print dss.utils.lines_to_dataframe()

              Bus1       Bus2   C0   C1  \
650632  rg60.1.2.3  632.1.2.3  1.6  3.4   

                                                  CMatrix Count EmergAmps  \
650632  [2.8, -0.6, -0.6, -0.6, 2.8, -0.6, -0.6, -0.6,...     1       600   

       Geometry Length LineCode  ...    \
650632            2000           ...     

                                                  RMatrix       Rg  Rho  \
650632  [0.0981333333333, 0.0401333333333, 0.040133333...  0.01805  100   

       Spacing Units       X0     X1  \
650632             5  1.93726  0.627   

                                                  XMatrix        Xg  Yprim  
650632  [0.2153, 0.0947, 0.0947, 0.0947, 0.2153, 0.094...  0.155081  [0.0]  

[1 rows x 27 columns]


In [152]:
a,b,c=sorted(['b','a','c'])

In [153]:
a

'a'

In [187]:
a={}
for k in range(10):
    a[k]={'test':k**2}

In [188]:
a

{0: {'test': 0},
 1: {'test': 1},
 2: {'test': 4},
 3: {'test': 9},
 4: {'test': 16},
 5: {'test': 25},
 6: {'test': 36},
 7: {'test': 49},
 8: {'test': 64},
 9: {'test': 81}}

In [189]:
b={}
for k in range(4,16):
    b[k]={ 'biscotte':k-2}

In [190]:
b

{4: {'biscotte': 2},
 5: {'biscotte': 3},
 6: {'biscotte': 4},
 7: {'biscotte': 5},
 8: {'biscotte': 6},
 9: {'biscotte': 7},
 10: {'biscotte': 8},
 11: {'biscotte': 9},
 12: {'biscotte': 10},
 13: {'biscotte': 11},
 14: {'biscotte': 12},
 15: {'biscotte': 13}}

In [194]:

def merge_dict(d1,d2):
    for k2,v2 in d2.iteritems():
        if d1.has_key(k2):
            d1[k2].update(v2)
        else:
            d1[k2]=v2
    return d1
            

In [195]:
merge_dict(a,b)

{0: {'test': 0},
 1: {'test': 1},
 2: {'test': 4},
 3: {'test': 9},
 4: {'biscotte': 2, 'test': 16},
 5: {'biscotte': 3, 'test': 25},
 6: {'biscotte': 4, 'test': 36},
 7: {'biscotte': 5, 'test': 49},
 8: {'biscotte': 6, 'test': 64},
 9: {'biscotte': 7, 'test': 81},
 10: {'biscotte': 8},
 11: {'biscotte': 9},
 12: {'biscotte': 10},
 13: {'biscotte': 11},
 14: {'biscotte': 12},
 15: {'biscotte': 13}}

In [185]:
a.update(b)

In [186]:
a

{0: {'test': 0},
 1: {'test': 1},
 2: {'test': 4},
 3: {'test': 9},
 4: {'biscotte': 2},
 5: {'biscotte': 3},
 6: {'biscotte': 4},
 7: {'biscotte': 5},
 8: {'biscotte': 6},
 9: {'biscotte': 7},
 10: {'biscotte': 8},
 11: {'biscotte': 9},
 12: {'biscotte': 10},
 13: {'biscotte': 11},
 14: {'biscotte': 12},
 15: {'biscotte': 13}}

In [168]:
b

{9: {'test': 729},
 10: {'test': 1000},
 11: {'test': 1331},
 12: {'test': 1728},
 13: {'test': 2197},
 14: {'test': 2744},
 15: {'test': 3375}}

In [197]:
class test:
   
    def __init__(self):
        self.a=1
        self.b=2
        self.c=3

In [198]:
tt=test()

In [199]:
dd={'a':3,'b':9}

In [200]:
for k,v in dd.iteritems():
    setattr(tt,k,v)

In [204]:
tt.c

3

In [205]:
mat=[[0,1,None],[1,0,None],[None,None,0]]

In [207]:
mat=np.array(mat)

In [208]:
mat

array([[0, 1, None],
       [1, 0, None],
       [None, None, 0]], dtype=object)

In [212]:
mat[mat!=None]

array([0, 1, 1, 0, 0], dtype=object)

In [214]:
pos=np.array([[1,1],[None,None],[4,4],[None,None]])

In [217]:
pos==np.array([None,None])

array([[False, False],
       [ True,  True],
       [False, False],
       [ True,  True]], dtype=bool)